In [ ]:
pip install PyPDF2

In [ ]:
pip install datasets

In [ ]:
pip install transformers

In [ ]:
!pip install kaggle

In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import datasets
import transformers

In [ ]:
#install all packages
!pip install tokenizer transformers scikit-learn pypdf==3.16.0 nltk pandarallel pandas==2.1.0 datasets #pyspark pyarrow

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d snehaanbhawal/resume-dataset

In [ ]:
#resume_data=pd.read_csv("kaggle resume dataset filepath") change it accordingly
import os
resume_data=pd.read_csv("Resume.csv")
resume_data.head()

In [ ]:
resume_data.info()

In [ ]:
resume_data.shape

In [ ]:
resume_data['Category'].unique()

In [ ]:
# check how many resume are available in every category.
resume_data['Category'].value_counts()

In [ ]:
len(resume_data['Category'].value_counts())

In [ ]:
# Visualize the distribution of categories
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

plt.figure(figsize=(15, 6))
sns.countplot(x=resume_data['Category'])
plt.xticks(rotation=90)
plt.title('Category Distribution')
plt.show()

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset

# Load the Hugging Face dataset for job descriptions
dataset = load_dataset('jacob-hugging-face/job-descriptions')

# Extract and comprehend 10-15 job descriptions
num_descriptions_to_extract = 15  # You can adjust this number as needed

# Loop through and print job descriptions
for i, description in enumerate(dataset['train']['job_description'][:num_descriptions_to_extract]):
    print(f"Job Description {i+1}:")
    print(description)
    print("="*50)  # Separator for readability


In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Tokenize and encode job descriptions and CVs
job_description= [...]  # List of job descriptions
cv_texts = [...]  # List of CV texts

In [ ]:
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, trainers
from transformers import AutoModel, AutoTokenizer
import torch

# Initialize a basic tokenizer
tokenizer = Tokenizer(models.BPE())

# Pre-tokenization
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Post-processing
tokenizer.decoder = decoders.ByteLevel()

# Train the tokenizer on your data
description_texts = ["your description text 1", "your description text 2"]
cv_texts = ["your CV text 1", "your CV text 2"]

all_texts = description_texts + cv_texts

trainer = trainers.BpeTrainer(special_tokens=["[PAD]", "[CLS]", "[SEP]", "[MASK]", "[UNK]"])
tokenizer.train_from_iterator(all_texts, trainer=trainer)

# Initialize the transformer model and tokenizer
model_name = "bert-base-uncased"  # Replace with the desired model name
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


# Encode the description and CV texts
description_encoded = tokenizer(description_texts, padding=True, truncation=True, return_tensors="pt", max_length=128)
cv_texts_encoded = tokenizer(cv_texts, padding=True, truncation=True, return_tensors="pt", max_length=128)

# Get embeddings
job_descriptions_embeddings = model(**description_encoded).last_hidden_state.mean(dim=1)
cv_texts_embeddings = model(**cv_texts_encoded).last_hidden_state.mean(dim=1)

# Now you have embeddings for your description and CV texts

In [ ]:
# Calculate cosine similarity

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Assuming you have two PyTorch tensors: job_descriptions_embeddings and cv_texts_embeddings
# Detach the tensors and convert them to NumPy arrays
job_descriptions_array = job_descriptions_embeddings.detach().numpy()
cv_texts_array = cv_texts_embeddings.detach().numpy()

# Calculate cosine similarity
similarity_matrix = cosine_similarity(job_descriptions_array, cv_texts_array)

In [ ]:
# Rank CVs based on similarity for each job description
top_candidates = []
for i, job_description in enumerate(job_descriptions):
    similarities = similarity_matrix[i]
    sorted_indices = similarities.argsort()[::-1]  # Sort in descending order
    top_candidates.append(sorted_indices[:5])  # Get top 5 candidates

In [ ]:
# Print top candidates for each job description
for i, candidates in enumerate(top_candidates):
    print(f"Job Description {i + 1} - Top Candidates:")
    for candidate_idx in candidates:
        print(f"CV Index: {candidate_idx}, Similarity: {similarity_matrix[i][candidate_idx]}")